### import

In [2]:
import pandas as pd
import glob
import numpy as np

### ファイル読み込み
fscore.csvかrecall.csvを使う

In [28]:
path = "victimrf/10/alltest/threshould/1/*_"

binlist = glob.glob(path+"bincount.csv")
scorelist = glob.glob(path+"recall.csv")

print(len(binlist), len(scorelist))

5 5


### F値またはRecallの平均値の計算

In [29]:
ave_score = []
ave_std = []
for i, score in enumerate(scorelist):
    scoretmp = pd.read_csv(score,header=0)
    #scoretmp[scoretmp.columns[i]] = None
    ave_score.append(scoretmp.mean())
    ave_std.append(scoretmp.std())

ave_score.append(pd.DataFrame(ave_score).mean())
ave_std.append(pd.DataFrame(ave_std).mean())

desc_score = (pd.DataFrame(ave_score)).describe()
desc_std = (pd.DataFrame(ave_std)).describe()

ave_score = pd.DataFrame(ave_score).round(3).astype(str)
ave_std = pd.DataFrame(ave_std).round(3).astype(str)

ave_merge = "\\shortstack{"+pd.DataFrame(ave_score)+"\\\\("+pd.DataFrame(ave_std)+")}"
ave_merge = ave_merge.loc[:,"0":"3"]
#for i, name in enumerate(ave_merge.columns):
#    ave_merge.loc[i,name] = None

ave_merge.index = ["V","W","X","Y","Z","平均"]
ave_merge.columns = ["A","B","C","D"]
display(ave_merge)


,A,B,C,D
V,\shortstack{0.007\\(0.018)},\shortstack{0.069\\(0.071)},\shortstack{0.006\\(0.029)},\shortstack{0.018\\(0.045)}
W,\shortstack{0.001\\(0.005)},\shortstack{0.222\\(0.202)},\shortstack{0.034\\(0.066)},\shortstack{0.787\\(0.196)}
X,\shortstack{0.02\\(0.063)},\shortstack{0.003\\(0.012)},\shortstack{0.01\\(0.036)},\shortstack{0.014\\(0.044)}
Y,\shortstack{0.0\\(0.0)},\shortstack{0.259\\(0.163)},\shortstack{0.764\\(0.276)},\shortstack{0.787\\(0.196)}
Z,\shortstack{0.001\\(0.005)},\shortstack{0.021\\(0.042)},\shortstack{0.0\\(0.0)},\shortstack{0.006\\(0.03)}
平均,\shortstack{0.006\\(0.018)},\shortstack{0.115\\(0.098)},\shortstack{0.163\\(0.081)},\shortstack{0.322\\(0.102)}


##### Recall、F値、FRR、FARの計算

In [30]:
#path = "cnnx/10/alltest/threshould/1/*_"
path = "victimrf/10/alltest/threshould/1/*_"

binlist = glob.glob(path+"bincount.csv")
scorelist = glob.glob(path+"recall.csv")

print(len(binlist), len(scorelist))

5 5


In [31]:
th = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
all_mean_std = []
for t in th:
    big_bool_list =[]
    for bin, score in zip(binlist,scorelist):
        bintmp = pd.read_csv(bin,header=None)
        scoretmp = pd.read_csv(score,header=0)
        bool_list =[]
        for i, name in enumerate(scoretmp.columns):
            df_bool = (scoretmp[name] > t) & (bintmp[i] == i)
            bool_list.append(df_bool.sum())
        big_bool_list.append(bool_list)
    df_th = pd.DataFrame(big_bool_list)

    """
    for i in range(len(df_th)):
        df_th.loc[i,i] = None
    """
    allcount = df_th.count()
    allcount = 100

    df_th = df_th.loc[:,"0":"3"]

    mean = df_th.mean().to_numpy()
    #std = np.round(df_th.std().to_numpy(),3)
    std = df_th.std().to_numpy()

    mean_std = []
    for m, s in zip(mean,std):
        mean_std.append(f"\shortstack{{{round(m,1)}\\\\({round(s,1)})}}")

    mean_std.append(f"\shortstack{{{round(mean.mean().mean(),1)}\\\\({round(std.mean(),1)})}}")
    
    #FRRの計算はここ
    #mean_std.append(f"\shortstack{{{round(1-(mean/allcount).mean(),3)}\\\\({round((mean/allcount).std(),3)})}}")
    #FARの計算はここ
    mean_std.append(f"\shortstack{{{round((mean/allcount).mean(),3)}\\\\({round((mean/allcount).std(),2)})}}")
    
    all_mean_std.append(mean_std)
df_all = pd.DataFrame(all_mean_std).T
df_all.columns = th

#df_all.index = ["A","B","C","D","E","F","G","H","I","J","K","平均","FRR"]
df_all.index = ["A","B","C","D","平均","FAR"]

display(df_all)

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
A,\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)}
B,\shortstack{10.4\\(14.4)},\shortstack{10.4\\(14.4)},\shortstack{10.4\\(14.4)},\shortstack{7.0\\(9.6)},\shortstack{4.0\\(5.7)},\shortstack{2.2\\(3.2)},\shortstack{1.2\\(2.2)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)}
C,\shortstack{17.6\\(39.4)},\shortstack{17.6\\(39.4)},\shortstack{17.6\\(39.4)},\shortstack{17.4\\(38.9)},\shortstack{16.2\\(36.2)},\shortstack{15.8\\(35.3)},\shortstack{15.0\\(33.5)},\shortstack{13.4\\(30.0)},\shortstack{8.4\\(18.8)}
D,\shortstack{38.4\\(52.6)},\shortstack{38.4\\(52.6)},\shortstack{38.4\\(52.6)},\shortstack{38.4\\(52.6)},\shortstack{37.2\\(50.9)},\shortstack{32.0\\(43.8)},\shortstack{28.8\\(39.4)},\shortstack{22.8\\(31.2)},\shortstack{16.8\\(23.0)}
平均,\shortstack{16.6\\(26.6)},\shortstack{16.6\\(26.6)},\shortstack{16.6\\(26.6)},\shortstack{15.7\\(25.3)},\shortstack{14.4\\(23.2)},\shortstack{12.5\\(20.6)},\shortstack{11.2\\(18.8)},\shortstack{9.0\\(15.3)},\shortstack{6.3\\(10.4)}
FAR,\shortstack{0.166\\(0.14)},\shortstack{0.166\\(0.14)},\shortstack{0.166\\(0.14)},\shortstack{0.157\\(0.14)},\shortstack{0.144\\(0.14)},\shortstack{0.125\\(0.13)},\shortstack{0.113\\(0.12)},\shortstack{0.09\\(0.1)},\shortstack{0.063\\(0.07)}


bincountの計算


In [27]:
#path = "rf/10/alltest/threshould/0/bincount/model*"
#path = "rf/10/alltest/threshould/1/bincount/model*"
#path = "cnnx/10/alltest/threshould/1/bincount/model*"
path = "victimcnn/10/alltest/threshould/1/bincount/model*"

bincountlist = glob.glob(path+".csv")

a = []
for i, bin in enumerate(bincountlist):
    a.append(np.loadtxt(bin,delimiter=",",dtype="int64"))
#display(a)
#a = np.array(a).mean(axis=1)
b = []
for i, bin in enumerate(a):
    b.append(pd.DataFrame(bin))
    display(bin)


b = []
for i in range(len(a)):
    for j in range(len(a)):
        b.append(a[j][i])
b = np.array(b)

b = np.split(b,5)
#display(b)
c = []
for i in range(len(a)):
    c.append(np.delete(b[i], i,axis=0))

c = np.array(c)

mean = pd.DataFrame(c.mean(axis=1))
std = pd.DataFrame(c.std(axis=1))

bin_mean_std = "\\shortstack{"+mean.round(1).astype(str)+"\\\\("+std.round(1).astype(str)+")}"
df_bin = pd.DataFrame(bin_mean_std)
#df_bin.columns = ["A","B","C","D","E","F","G","H","I","J","K"]
#df_bin.index = ["A","B","C","D","E","F","G","H","I","J","K"]

#display(df_bin)

array([[  0,   0,   0,   0,   0,   0,  18,   0,   0,  82],
       [  1,  39,   0,   0,   0,   0,   1,   0,   0,  59],
       [  2,   0,   7,   0,   0,   0,  45,   0,   0,  46],
       [  1,   0,   0,   0,   0,   0,   8,   2,   0,  89],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   5,  95,   0,   0,   0,   0],
       [  0,   0,   0,   2,   0,   0,  98,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   1,  95,   0,   3],
       [  0,   1,   0,   0,   2,   0,   0,   0,  97,   0],
       [  1,   0,   0,   0,   0,   0,   3,   1,   0,  95]])

array([[  5,   0,   1,   0,   0,   0,  29,   0,   6,  59],
       [ 10,   3,   0,   0,   6,   0,   0,   0,  19,  62],
       [ 13,   0,   0,  12,   0,   0,  15,   0,   0,  60],
       [ 37,   0,   0,   0,   1,   0,  53,   1,   4,   4],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   5,  95,   0,   0,   0,   0],
       [  0,   0,   0,   2,   0,   0,  98,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   1,  95,   0,   3],
       [  0,   1,   0,   0,   2,   0,   0,   0,  97,   0],
       [  1,   0,   0,   0,   0,   0,   3,   1,   0,  95]])

array([[  2,   2,   0,   0,   0,   0,   3,   0,  38,  55],
       [  8,   0,   0,   0,   0,   0,   0,   0,   1,  91],
       [ 70,   0,   0,   0,   0,   0,  19,   0,   0,  11],
       [ 10,   0,   0,   2,   0,   0,  42,   0,   1,  45],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   5,  95,   0,   0,   0,   0],
       [  0,   0,   0,   2,   0,   0,  98,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   1,  95,   0,   3],
       [  0,   1,   0,   0,   2,   0,   0,   0,  97,   0],
       [  1,   0,   0,   0,   0,   0,   3,   1,   0,  95]])

array([[  0,   0,   0,   0,   0,   0,  69,   0,   0,  31],
       [ 13,   8,   0,   0,   0,   1,  37,   1,  11,  29],
       [ 21,   0,   0,   2,   0,   2,  56,   0,   8,  11],
       [ 37,   0,   0,   0,   1,   0,  53,   1,   4,   4],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   5,  95,   0,   0,   0,   0],
       [  0,   0,   0,   2,   0,   0,  98,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   1,  95,   0,   3],
       [  0,   1,   0,   0,   2,   0,   0,   0,  97,   0],
       [  1,   0,   0,   0,   0,   0,   3,   1,   0,  95]])

array([[  0,   0,   0,   0,   1,   0,   5,   0,   0,  94],
       [  1,   3,   0,   0,   1,   0,   3,   0,   1,  91],
       [  0,   0,   0,   0,   0,   0,   1,   0,   0,  99],
       [  2,   0,   0,   0,   0,   0,   0,   0,   0,  98],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   5,  95,   0,   0,   0,   0],
       [  0,   0,   0,   2,   0,   0,  98,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   1,  95,   0,   3],
       [  0,   1,   0,   0,   2,   0,   0,   0,  97,   0],
       [  1,   0,   0,   0,   0,   0,   3,   1,   0,  95]])

### ↓模倣者だと使わないかも

In [9]:
#path = "rf/10/alltest/threshould/0/bincount/model*"
#path = "rf/10/alltest/threshould/1/bincount/model*"
#path = "cnnx/10/alltest/threshould/1/bincount/model*"
path = "victimfcnn/10/alltest/threshould/1/bincount/model*"

bincountlist = glob.glob(path+".csv")

a = []
for i, bin in enumerate(bincountlist):
    a.append(np.loadtxt(bin,delimiter=",",dtype="int64"))
#display(a)
#print(np.shape(a))


b = []
for i in range(len(a)):
    for j in range(len(a)):
        b.append(a[j][i])
b = np.array(b)

b = np.split(b,5)
display(b)
c = []
for i in range(len(a)):
    c.append(np.delete(b[i], i,axis=0))

c = np.array(c)

mean = pd.DataFrame(c.mean(axis=1))
std = pd.DataFrame(c.std(axis=1))

bin_mean_std = "\\shortstack{"+mean.round(1).astype(str)+"\\\\("+std.round(1).astype(str)+")}"
df_bin = pd.DataFrame(bin_mean_std)
#df_bin.columns = ["A","B","C","D","E","F","G","H","I","J","K"]
#df_bin.index = ["A","B","C","D","E","F","G","H","I","J","K"]

display(df_bin)

[array([[ 3,  0,  0,  0,  0,  0, 17,  0,  0, 80],
        [ 0,  0,  0,  0,  0,  0,  2,  0, 36, 62],
        [ 0,  0,  0,  0,  0,  8,  0,  0, 78, 14],
        [ 0,  0,  0,  0,  0,  1,  1,  4, 14, 80],
        [ 0,  2,  0,  0,  0,  1, 13,  0,  1, 83]]),
 array([[ 0, 32,  0,  0,  0,  0,  2,  0,  1, 65],
        [ 0,  2,  0,  0,  6,  1,  4,  0, 74, 13],
        [12, 30,  0,  0,  0,  0, 11,  0, 43,  4],
        [ 0,  0,  0,  0,  9,  1,  2,  5, 69, 14],
        [ 0,  9,  0,  0,  0,  0,  7,  0, 38, 46]]),
 array([[ 0,  0,  0,  0,  0,  0, 14,  0,  0, 86],
        [ 0,  0,  0, 33,  0,  0,  0,  0,  0, 67],
        [26,  0,  0,  0,  0, 21, 17,  0,  0, 36],
        [ 0,  0,  0,  0,  4, 23,  1,  0, 12, 60],
        [ 0,  6,  0,  0,  0,  0,  5,  1,  5, 83]]),
 array([[ 0,  0,  0,  4,  0,  0,  0,  2,  0, 94],
        [21,  3,  0,  0,  1,  0, 19,  0,  6, 50],
        [ 3,  0,  0, 28,  0,  3,  2,  0,  0, 64],
        [21,  3,  0,  0,  1,  0, 19,  0,  6, 50],
        [16,  0,  0,  0,  0,  0, 19,  0,  0,

,0,1,2,3,4,5,6,7,8,9
0,\shortstack{0.0\\(0.0)},\shortstack{0.5\\(0.9)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{2.5\\(3.2)},\shortstack{4.0\\(5.2)},\shortstack{1.0\\(1.7)},\shortstack{32.2\\(29.2)},\shortstack{59.8\\(27.6)}
1,\shortstack{3.0\\(5.2)},\shortstack{17.8\\(13.6)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{2.2\\(3.9)},\shortstack{0.2\\(0.4)},\shortstack{5.5\\(3.8)},\shortstack{1.2\\(2.2)},\shortstack{37.8\\(24.3)},\shortstack{32.2\\(24.5)}
2,\shortstack{0.0\\(0.0)},\shortstack{1.5\\(2.6)},\shortstack{0.0\\(0.0)},\shortstack{8.2\\(14.3)},\shortstack{1.0\\(1.7)},\shortstack{5.8\\(10.0)},\shortstack{5.0\\(5.5)},\shortstack{0.2\\(0.4)},\shortstack{4.2\\(4.9)},\shortstack{74.0\\(10.8)}
3,\shortstack{10.0\\(8.7)},\shortstack{0.8\\(1.3)},\shortstack{0.0\\(0.0)},\shortstack{8.0\\(11.7)},\shortstack{0.2\\(0.4)},\shortstack{0.8\\(1.3)},\shortstack{10.0\\(9.0)},\shortstack{0.5\\(0.9)},\shortstack{1.5\\(2.6)},\shortstack{68.2\\(16.0)}
4,\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{100.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)},\shortstack{0.0\\(0.0)}
